In [1]:
from posenet.model import PoseNetDinoImproved
from nflownet.model import NFlowNet
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from dataset.tartanair import TartanAirDataset
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
set_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
def load_posenet(posenet_path):
    # Initialize FlowNet model
    posenet = PoseNetDinoImproved().to(device)
    
    # Load weights from .pth file
    checkpoint = torch.load(posenet_path, map_location=device)
    
    # Handle different save formats
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    elif 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
    else:
        state_dict = checkpoint
    
    # Load state dict into model
    posenet.load_state_dict(state_dict)
    posenet.train()  # Set to evaluation mode
    
    return posenet

In [3]:
def load_nflownet(nflownet_path):
    # Initialize FlowNet model
    nflownet = NFlowNet().to(device)
    
    # Load weights from .pth file
    checkpoint = torch.load(nflownet_path, map_location=device)
    
    # Handle different save formats
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    elif 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
    else:
        state_dict = checkpoint
    
    # Load state dict into model
    nflownet.load_state_dict(state_dict)
    nflownet.eval()  # Set to evaluation mode
    
    return nflownet

In [4]:
posenet_path = r"D:\KOC UNIVERSITY\COMP447\trainedmodels\posenet\posenet2.pth"
nflownet_path = r"D:\KOC UNIVERSITY\COMP447\trainedmodels\nflownet\nflownet_final.pth"

posenet = load_posenet(posenet_path)
nflownet = load_nflownet(nflownet_path)

Using cache found in C:\Users\emircan/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\emircan/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\emircan/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\emircan/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
C:\Users\emircan\AppData\Local\Temp\ipykernel_19116\4256205414.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from nflownet.utils import compute_image_gradients
import kornia
from torchvision import transforms

class CheiralityLayer(nn.Module):
    def __init__(self, posenet, nflownet):
        super().__init__()
        self.posenet = posenet
        self.nflownet = nflownet
        self.nflownet.eval()  # freeze NFlowNet
        for p in self.nflownet.parameters():
            p.requires_grad = False
            

    def construct_A_B(self, H, W, device):
        y, x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
        x = x.to(device).float()
        y = y.to(device).float()

        # A and B: [H, W, 2, 3]
        A = torch.stack([
            -torch.ones_like(x), torch.zeros_like(x), x,
            torch.zeros_like(y), -torch.ones_like(y), y
        ], dim=-1).reshape(H, W, 2, 3)

        B = torch.stack([
            x * y, -(x ** 2 + 1), y,
            y ** 2 + 1, -x * y, -x
        ], dim=-1).reshape(H, W, 2, 3)

        # Reshape to [H*W, 2, 3]
        A = A.view(-1, 2, 3)
        B = B.view(-1, 2, 3)

        return A, B

    def cheirality_loss(self, img_pair_shape, pose, grad_dirs, normal_flow, device):
        B, _, H, W = img_pair_shape
        V, W_ = pose[:, :, :3], pose[:, :, 3:]  # [B, 1, 3]

        A, B_mat = self.construct_A_B(H, W, device)  # [H*W, 2, 3]

        # Gradient directions
        grad_x, grad_y = grad_dirs  # each: [B, H, W]
        gx = torch.stack([grad_x, grad_y], dim=-1).view(B, H*W, 2)  # [B, H*W, 2]
        gx_unit = F.normalize(gx, dim=-1)  # [B, H*W, 2]

        # Normal flow: [B, 2, H, W] -> [B, H*W, 2]
        nf = normal_flow.permute(0, 2, 3, 1).reshape(B, H*W, 2)

        # Compute scalar projection of normal flow onto gradient direction
        nf_scalar = torch.sum(nf * gx_unit, dim=-1)  # [B, H*W]

        # gA = gx · A → [B, H*W, 3]
        gA = torch.einsum('bpi,pij->bpj', gx_unit, A)

        # gB = gx · B → [B, H*W, 3]
        gB = torch.einsum('bpi,pij->bpj', gx_unit, B_mat)

        # term1 = gA @ V → [B, H*W]
        term1 = torch.sum(gA * V, dim=-1)

        # term2 = nf_scalar - (gB @ Omega) → [B, H*W]
        term2 = nf_scalar - torch.sum(gB * W_, dim=-1)

        # rho = term1 * term2
        rho = term1 * term2  # [B, H*W]
        # Penalize negative values of rho
        loss = F.gelu(-rho).mean()
        return loss

    def refine_pose(self, img_pair_shape, Pec, grad_dirs, normal_flow):
        B = Pec.shape[0]
        Per = Pec.clone().detach().requires_grad_(True)

        optimizer = torch.optim.LBFGS([Per], max_iter=100, line_search_fn="strong_wolfe")

        def closure():
            optimizer.zero_grad()
            loss = self.cheirality_loss(img_pair_shape, Per, grad_dirs, normal_flow, Pec.device)
            loss.backward() 
            torch.nn.utils.clip_grad_norm_([Per], max_norm=100)
            return loss

        optimizer.step(closure)

        # Cheirality loss after refinement
        loss_cheirality = self.cheirality_loss(img_pair_shape, Per, grad_dirs, normal_flow, Pec.device)

        # Compute dL/dPer (gradient of cheirality loss wrt Per)
        grad_cheirality = grad(loss_cheirality, Per, create_graph=True)[0]  # shape: [B, 6]

        return Per.detach().requires_grad_(), grad_cheirality

    def upper_level_loss(self, img_pair_shape, Pec, Per, n_flow_pred, grad_dirs, device):
        B, _, H, W = img_pair_shape
        A, B_mat = self.construct_A_B(H, W, device)  # [H*W, 2, 3]

        grad_x, grad_y = grad_dirs  # each [B, H, W]

        # Stack and reshape gradient directions: [B, H, W, 2] -> [B, H*W, 2]
        g_x = torch.stack([grad_x, grad_y], dim=-1).view(B, H*W, 2)
        gx_unit = F.normalize(g_x, dim=-1)  # normalize gradient directions

        # Reshape predicted normal flow vector to [B, H*W, 2]
        n_flow_vec = n_flow_pred.permute(0, 2, 3, 1).reshape(B, H*W, 2)  # [B, H*W, 2]

        # Project predicted normal flow vector along gradient directions (scalar)
        n_flow_scalar = torch.sum(n_flow_vec * gx_unit, dim=-1)  # [B, H*W]

        # Pose splits (translation and rotation)
        V_r, Omega_r = Per[:,:, :3], Per[:,:, 3:]  # [B, 3]
        V_c, Omega_c = Pec[:,:, :3], Pec[:,:, 3:]  # [B, 3]

        # Project A and B matrices along gradient directions
        g_dot_A = torch.einsum("bpi,pij->bpj", gx_unit, A)      # [B, H*W, 3]
        g_dot_B = torch.einsum("bpi,pij->bpj", gx_unit, B_mat)  # [B, H*W, 3]

        # Compute derotation term: (gx · B) · Omega_r
        g_dot_B_Omega_r = torch.sum(g_dot_B * Omega_r.unsqueeze(1), dim=-1)  # [B, H*W]

        # Compute translational term: (gx · A) · V_r
        g_dot_A_V_r = torch.sum(g_dot_A * V_r.unsqueeze(1), dim=-1)  # [B, H*W]

        # Compute denominator of depth scale: n_x - (gx · B) · Omega_r
        denom = n_flow_scalar - g_dot_B_Omega_r  # [B, H*W]
        eps = 1e-6
        depth_scale = g_dot_A_V_r / (denom + eps)  # [B, H*W]

        # Compute terms for coarse pose Pec
        g_dot_A_V_c = torch.sum(g_dot_A * V_c.unsqueeze(1), dim=-1)  # [B, H*W]
        g_dot_B_Omega_c = torch.sum(g_dot_B * Omega_c.unsqueeze(1), dim=-1)  # [B, H*W]

        # Predict normal flow scalar from coarse pose Pec using depth scale
        n_flow_pred_from_pose = (g_dot_A_V_c / (depth_scale + eps)) - g_dot_B_Omega_c  # [B, H*W]

        # Compute MSE loss between predicted scalar normal flow and model predicted scalar normal flow
        n_flow_pred_from_pose = n_flow_pred_from_pose.squeeze(1)  # now shape [1, 307200]

        loss = F.mse_loss(n_flow_pred_from_pose, n_flow_scalar)
        return loss


    def forward(self, img_pair):
        img1, img2,img3,img4,img5,img6 = torch.chunk(img_pair, chunks=6, dim=1)

        # Remove the singleton dimension (1) at dim=1
        img1 = img1.squeeze(1)  # shape: [1, 3, 480, 640]
        img2 = img2.squeeze(1)
        img3 = img3.squeeze(1)
        img4 = img4.squeeze(1)
        img5 = img5.squeeze(1)
        img6 = img6.squeeze(1)  # [1, 3, 480, 640]
        # Resize & normalize manually
        def preprocess(img):
            img = F.interpolate(img, size=(224, 224), mode='bilinear', align_corners=False)  # [B, 3, 224, 224]
            mean = torch.tensor([0.485, 0.456, 0.406], device=img.device).view(1, 3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225], device=img.device).view(1, 3, 1, 1)
            img = (img - mean) / std
            return img

        a_ = preprocess(img1)  # [B, 3, 224, 224]
        b_ = preprocess(img2)
        c_ = preprocess(img3)  # [B, 3, 224, 224]
        d_ = preprocess(img4)
        e_ = preprocess(img5)  # [B, 3, 224, 224]
        f_ = preprocess(img6)  # [B, 3, 224, 224]

        pose_img = torch.stack([a_, b_,c_,d_,e_,f_], dim=1) 
    
        translation, rotation = self.posenet(pose_img)
        print("Posenet Çıktıları:")
        print("Posenet Çıktıları Translation:")

        translation = translation[:,:1,:]
        print(translation)
        rotation = rotation[:,:1,:]
        print("Posenet Çıktıları Quaternion:")
        print(rotation)
        q_flat = rotation.reshape(-1, 4)
        # Convert to rotation vector (axis-angle): [B * T, 3]
        rotvec_flat = kornia.geometry.quaternion_to_axis_angle(q_flat)
        print("Posenet Çıktıları Rotation:")
        print(rotvec_flat)
        # reshape: [B, T, 3]
        rotation = rotvec_flat.view(rotation.shape[0], rotation.shape[1], 3)
        Pec = torch.cat([translation, rotation], dim=-1) 
        img_pair = img_pair[:,:2]
        img_pair = img_pair.view(1, 2 * 3, 480, 640)
        nflow_pred = self.nflownet(img_pair)
        nflow_pred = nflow_pred[:, :, 1:-1, 1:-1]  # [1, 2, 478, 638]
        gray = nflow_pred[:, :3].mean(1, keepdim=True)
        grad_dirs = compute_image_gradients(gray)  # [B, 2, H, W]
        img_rand = torch.rand(1, 6, 478, 638)
        img_pair_shape = img_rand.shape
        # Lower-level: refine using cheirality
        Per, dL_dPer = self.refine_pose(img_pair_shape, Pec, grad_dirs, nflow_pred)       
        print("Refined Pose:")
        print(Per)
        upper_loss = self.upper_level_loss(img_pair_shape, Pec, Per, nflow_pred, grad_dirs,img_pair.device)
        
        # ∂L_upper / ∂Pec + ∂L_upper / ∂Per * ∂Per / ∂Pec
        dL_dPec_upper = grad(upper_loss, Pec, retain_graph=True, create_graph=True)[0]
        dL_dPer_upper = grad(upper_loss, Per, retain_graph=True, create_graph=True)[0]

        # ∂L_total / ∂Pec = ∂L_upper / ∂Pec - dL_dPer_upper ⊙ d_cheirality_loss / d_Per
        total_grad = dL_dPec_upper - torch.autograd.grad((dL_dPer * dL_dPer_upper).sum(), Pec, retain_graph=True)[0]

        # Kayıp gibi davranarak backward yap
        dummy_loss = (Pec * total_grad.detach()).sum()
        return dummy_loss
           

In [6]:
cheirality_layer = CheiralityLayer(nflownet=nflownet,posenet=posenet).to(device)

In [7]:
# Dataset and DataLoader
dataset = TartanAirDataset(root_dir="D:/KOC UNIVERSITY/COMP447/tartanair_dataset",seq_len=6)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

: 

In [ ]:
optimizer = optim.Adam(posenet.parameters(), lr=1e-4)

num_epochs = 1
for epoch in range(num_epochs):
    total_loss = 0.0
    
    for images, translation, rotation in train_loader:
        print("Ground Truth Translation:")
        print(translation[:,:1,:])
        print("Ground Truth Quaternion:")
        rotation = rotation[:,:1,:]
        print(rotation)
        q_flat = rotation.reshape(-1, 4)
        # Convert to rotation vector (axis-angle): [B * T, 3]
        rotvec_flat = kornia.geometry.quaternion_to_axis_angle(q_flat)
        print("Ground Truth Rotation:")
        print(rotvec_flat)
        images = images.to(device)
        loss = cheirality_layer(images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print(loss.item())
    print(f"Epoch {epoch}, Loss: {total_loss / len(train_loader)}")
        

Ground Truth Translation:
tensor([[[ 0.0387, -0.2009,  0.0501]]])
Ground Truth Quaternion:
tensor([[[ 6.7457e-09, -4.7890e-08, -3.5645e-03,  9.9999e-01]]])
Ground Truth Rotation:
tensor([[-1.5045e-07, -1.1198e-02,  3.1416e+00]])
Posenet Çıktıları:
Posenet Çıktıları Translation:
tensor([[[-0.0020,  0.0042, -0.0034]]], device='cuda:0',
       grad_fn=<SliceBackward0>)
Posenet Çıktıları Quaternion:
tensor([[[-0.4018, -0.3338, -0.0230, -0.8524]]], device='cuda:0',
       grad_fn=<SliceBackward0>)
Posenet Çıktıları Rotation:
tensor([[0.8438, 0.0581, 2.1545]], device='cuda:0',
       grad_fn=<AsStridedBackward0>)
